In [1]:
import pandas as pd

from ast import literal_eval
from IPython.display import display
from ipywidgets import interact, widgets
from rdkit.Chem import Draw, Mol, PandasTools

from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.molSize = (600, 600)

In [6]:
root = '/home/user/AutoSOM/output/'

In [ ]:
substrates = PandasTools.LoadSDF(root + "substrates.sdf")
metabolites = PandasTools.LoadSDF(root + "metabolites.sdf")
data = pd.merge(
    substrates,
    metabolites,
    on=[
        "ID",
        "substrate_id",
        "metabolite_id",
        "soms",
        "annotation_rule",
    ],
    how="inner",
    suffixes=("_substrate", "_metabolite"),
)
data["soms"] = data["soms"].apply(literal_eval)

In [ ]:
@interact(i=widgets.IntSlider(min=0, max=len(data) - 1, step=1))
def f(i):

    row = data.iloc[i]

    substrate = Mol(row.ROMol_substrate)
    substrate.Compute2DCoords()
    for atom in substrate.GetAtoms():
        atom.SetIntProp("atomNote", atom.GetIdx())

    metabolite = Mol(row.ROMol_metabolite)
    metabolite.Compute2DCoords()
    for atom in metabolite.GetAtoms():
        atom.SetIntProp("atomNote", atom.GetIdx())

    print(f"Substrate: {row.substrate_id}")
    print(f"Metabolite: {row.metabolite_id}")
    print(f"Predicted SoMs: {row.soms}")
    print(f"Annotation rule: {row.annotation_rule}")

    display(
        Draw.MolsToGridImage(
            [substrate, metabolite],
            molsPerRow=2,
            subImgSize=(500, 500),
            legends=["substrate", "metabolite"],
            highlightAtomLists=[row.soms, []],
            highlightBondLists=[],
        )
    )